In [1]:
#Liste des imports

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


In [12]:
#Récupérer les données 

label = np.load("Label.npy")
targetBac = np.load("targetBac.npy")
targetK7 =  np.load("targetK7.npy")

""""
#On les convertis en tensors
labelTensor = torch.from_numpy(label)
targetBacTensor = torch.from_numpy(targetBac)
targetK7Tensor = torch.from_numpy(targetK7)
""""

In [15]:
#Paramètres

input_size= len(label[0])
hidden_dim = 100
n_layers = 1
output_size = len(label[0])

batch_size = 100
num_epochs = 5
n_iter = epochs * (len(targetBac) / batch_size)
n_iter = int(n_iter)


In [16]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)
        # FC Layer
        self.fc = nn.Linear(hidden_dim, output_size)


    def forward(self, x):
        batch_size = x.size(0)

        # Initializing hidden state for thr first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining output
        out, hidden = self.rnn(x, hidden)

        # Reshaping the outputs such that it can be fit into the FC Layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch .zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

In [17]:
# Training Run
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    input_seq.to(device)
    output, hidden = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and alculates gradients
    optimizer.step()

    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

ValueError: too many values to unpack (expected 2)

tensor([2018,    8,    1,  360], dtype=torch.int32)
